In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pickle
import gzip
import time
from warnings import filterwarnings
filterwarnings('ignore')
from _utils import utils, transforming, filling

data = ['../BigContest_data/' + file for file in os.listdir('../BigContest_data')]

In [2]:
concat_log = pd.read_csv('../preprocessed/concat_log.csv')

# log_data에 대한 feature
## log_data의 행동 횟수(비율)
- 여러번의 is_applied 전후의 행동 비중 살피기

In [ ]:
concat_log['event'].unique()

In [4]:
valid_actions = ['UseLoanManage','CompleteIDCertification','UsePrepayCalc','UseDSRCalc','GetCreditInfo','is_applied_Y']
cond = concat_log['event'].isin(valid_actions)
concat_log_cond = concat_log[cond]

In [21]:
# 주의 : 30분(?) 가량 걸림
start = time.time()
freq_action = concat_log_cond.groupby('user_id').apply(transforming.freq_count)
print((time.time() - start)/60,'분')

32.642743690808615 분


## log_data의 시간 feature

In [ ]:
# 예시
temp = concat_log_cond.query('user_id==879693')
display(temp)
print(transforming.timedelta_static(temp))

In [14]:
print((pd.to_datetime('2022-05-20 16:33:04') - pd.to_datetime('2022-05-13 13:24:33')).components)
7*24 +3+ 8/60+31/3600

Components(days=7, hours=3, minutes=8, seconds=31, milliseconds=0, microseconds=0, nanoseconds=0)


171.14194444444445

In [ ]:
temp = concat_log_cond.query('user_id==1')
display(temp)
print(transforming.timedelta_static(temp))

In [22]:
start = time.time()
concat_log_cond['temp'] = concat_log_cond['event'] + '/'+ concat_log_cond['timestamp'] 
user_time_delta = concat_log_cond.groupby('user_id')['temp'].agg(transforming.timedelta_static_wrapper)
processing_seconds = time.time() - start
print(processing_seconds/60,'분')

9.904152647654216 분


In [24]:
user_time_delta_df = user_time_delta.str.split(pat=',',expand=True)
categories_conti = pd.Series(['max','min','mean','std','quantile_first','quantile_second','quantile_third'], dtype='str')
user_time_delta_df.columns = ('timedelta_'+ categories_conti).tolist()
user_time_delta_df = user_time_delta_df.applymap(utils.as_float)

## 행동특성과 시간특성 concat

In [28]:
log_clustering = pd.concat([freq_action,user_time_delta_df],axis=1)
utils.save_pickle(log_clustering, '../preprocessed/log_clustering.pickle')

The Object saved in ../preprocessed/log_clustering.pickle.


## log 행동특성 월간 평균 횟수

In [123]:
concat_log_valid = concat_log[concat_log['event'].isin(['UseLoanManage','CompleteIDCertification','UsePrepayCalc','UseDSRCalc','GetCreditInfo'])]
concat_log_valid['month'] = pd.to_datetime(concat_log_valid['timestamp']).dt.month

grouped = concat_log_valid.groupby(['user_id','event','month']).count()
grouped_stacked = grouped.unstack(level=2).unstack(level=1)
first_lev_col = np.array(grouped_stacked.columns.tolist())[:,1]
second_lev_col = np.array(grouped_stacked.columns.tolist())[:,2]
new_col = ['month_'+x+'_'+y+'_count' for x,y in zip(first_lev_col, second_lev_col)]
grouped_stacked.columns = new_col

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
action_counts = pd.DataFrame()
grouped_stacked = grouped_stacked.fillna(0)
action_counts['CompleteIDCertification_count'] = (grouped_stacked.iloc[:,0] + grouped_stacked.iloc[:,5] + grouped_stacked.iloc[:,10])/3
action_counts['GetCreditInfo_count'] = (grouped_stacked.iloc[:,1] + grouped_stacked.iloc[:,6] + grouped_stacked.iloc[:,11])/3
action_counts['UseDSRCalc_count'] = (grouped_stacked.iloc[:,2] + grouped_stacked.iloc[:,7] + grouped_stacked.iloc[:,12])/3
action_counts['UseLoanManage_count'] = (grouped_stacked.iloc[:,3] + grouped_stacked.iloc[:,8] + grouped_stacked.iloc[:,13])/3
action_counts['UsePrepayCalc_count'] = (grouped_stacked.iloc[:,4] + grouped_stacked.iloc[:,9] + grouped_stacked.iloc[:,14])/3


In [ ]:
action_counts.head()

In [ ]:
log_action_clustering = pd.concat([log_clustering,action_counts],axis=1)
log_action_clustering.head()

In [135]:
utils.save_pickle(log_action_clustering, '../preprocessed/log_action_clustering.pickle')

The Object saved in ../preprocessed/log_action_clustering.pickle.
